In [ ]:
%%writefile evaluate_fitness_batch.py
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load dataset
df = pd.read_csv("Crop_recommendation.csv")
X = df.drop(columns="label")  # Ensure all features are included
y = df["label"]

# Read population (batch of chromosomes) from a file
population = np.loadtxt("population_batch.txt", delimiter=",", dtype=int)
print(f"Population Shape: {population.shape}")

# Validate chromosome length
if population.shape[1] != X.shape[1]:
    raise ValueError(f"Chromosome length {population.shape[1]} does not match the number of features {X.shape[1]}")

# Initialize list to store fitness values
fitness_results = []

# Loop through each chromosome in the population
for chromosome_id, chromosome in enumerate(population, start=1):
    print(f"Chromosome {chromosome_id}: {chromosome}")

    # Get selected features
    selected_features = np.where(chromosome == 1)[0]
    print("Selected Features:", selected_features)

    # Handle case where no features are selected
    if len(selected_features) == 0:
        print(f"Chromosome {chromosome_id}: No features selected. Assigning fitness 0.")
        fitness_results.append(0.0)
        continue

    # Select features based on chromosome
    X_selected = X.iloc[:, selected_features]

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42)

    # Train the model
    rf = RandomForestClassifier(n_estimators=50, random_state=42, criterion="entropy")
    rf.fit(X_train, y_train)

    # Predict and calculate accuracy
    y_pred = rf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Chromosome {chromosome_id} Accuracy:", accuracy)

    # Store fitness result
    fitness_results.append(accuracy)

# Save all fitness results to a file
np.savetxt("fitness_results_batch.txt", fitness_results, fmt="%.4f")
print("Fitness results saved.")


Overwriting evaluate_fitness_batch.py


In [ ]:
%%writefile cudamain.cu

#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <curand_kernel.h>

#define POP_SIZE 80
#define CHROMOSOME_LENGTH 7
#define NUM_GENERATIONS 50
#define MUTATION_RATE 0.01f
#define MAX_STAGNATION 3
#define BLOCK_SIZE 256

typedef struct {
    int id;
    int genes[CHROMOSOME_LENGTH];
    double fitness;
} Individual;

// CUDA kernel for random chromosome generation
__global__ void generate_population_kernel(int* population, curandState* states, int pop_size, int chromosome_length) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < pop_size) {
        curandState localState = states[idx];
        for (int i = 0; i < chromosome_length; i++) {
            population[idx * chromosome_length + i] = curand(&localState) % 2;
        }
        states[idx] = localState;
    }
}

// CUDA kernel for mutation
__global__ void mutation_kernel(int* population, curandState* states, float mutation_rate, int pop_size, int chromosome_length) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < pop_size * chromosome_length) {
        int chr_idx = idx / chromosome_length;
        int gene_idx = idx % chromosome_length;

        curandState localState = states[chr_idx];
        float r = curand_uniform(&localState);
        if (r < mutation_rate) {
            population[idx] = 1 - population[idx];
        }
        states[chr_idx] = localState;
    }
}

// CUDA kernel for crossover
__global__ void crossover_kernel(int* population, int* new_population, int pop_size, int chromosome_length) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < pop_size / 2) {
        int parent1_idx = 2 * idx;
        int parent2_idx = 2 * idx + 1;

        int swap_start = chromosome_length - 3;

        // Pointers to parents and offspring
        int* parent1 = &population[parent1_idx * chromosome_length];
        int* parent2 = &population[parent2_idx * chromosome_length];
        int* offspring1 = &new_population[parent1_idx * chromosome_length];
        int* offspring2 = &new_population[parent2_idx * chromosome_length];

        // Perform three-point crossover
        for (int i = 0; i < chromosome_length; i++) {
            if (i < swap_start) {
                offspring1[i] = parent1[i];
                offspring2[i] = parent2[i];
            } else {
                offspring1[i] = parent2[i];
                offspring2[i] = parent1[i];
            }
        }
    }
}

// Function to initialize random states
__global__ void init_random_states(curandState* states, unsigned long seed) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < POP_SIZE) {
        curand_init(seed, idx, 0, &states[idx]);
    }
}

int main() {
    // Allocate device memory
    int *d_population, *d_new_population;
    curandState *d_states;
    cudaMalloc(&d_population, POP_SIZE * CHROMOSOME_LENGTH * sizeof(int));
    cudaMalloc(&d_new_population, POP_SIZE * CHROMOSOME_LENGTH * sizeof(int));
    cudaMalloc(&d_states, POP_SIZE * sizeof(curandState));

    // Initialize random states
    int blocks = (POP_SIZE + BLOCK_SIZE - 1) / BLOCK_SIZE;
    init_random_states<<<blocks, BLOCK_SIZE>>>(d_states, time(NULL));
    cudaDeviceSynchronize();

    // Generate initial population
    generate_population_kernel<<<blocks, BLOCK_SIZE>>>(d_population, d_states, POP_SIZE, CHROMOSOME_LENGTH);
    cudaDeviceSynchronize();

    // Allocate host memory
    Individual* population = (Individual*)malloc(POP_SIZE * sizeof(Individual));
    Individual* new_population = (Individual*)malloc(POP_SIZE * sizeof(Individual));
    int* h_population = (int*)malloc(POP_SIZE * CHROMOSOME_LENGTH * sizeof(int));
    cudaMemcpy(h_population, d_population, POP_SIZE * CHROMOSOME_LENGTH * sizeof(int), cudaMemcpyDeviceToHost);

    // Initialize population structs
    for (int i = 0; i < POP_SIZE; i++) {
        population[i].id = i + 1;
        memcpy(population[i].genes, &h_population[i * CHROMOSOME_LENGTH], CHROMOSOME_LENGTH * sizeof(int));
    }

    double best_fitness = -1.0;
    int stagnation_count = 0;

    for (int generation = 0; generation < NUM_GENERATIONS; generation++) {
        printf("\n=== Generation %d ===\n", generation);

        // Write population to file
        FILE* file = fopen("population_batch.txt", "w");
        if (!file) {
            perror("Error opening batch file");
            exit(EXIT_FAILURE);
        }
        for (int i = 0; i < POP_SIZE; i++) {
            for (int j = 0; j < CHROMOSOME_LENGTH; j++) {
                fprintf(file, "%d", population[i].genes[j]);
                if (j < CHROMOSOME_LENGTH - 1) fprintf(file, ",");
            }
            fprintf(file, "\n");
        }
        fclose(file);

        // Execute Python script
        system("python3 evaluate_fitness_batch.py");

        // Read fitness results
        file = fopen("fitness_results_batch.txt", "r");
        if (!file) {
            perror("Error opening fitness results batch file");
            exit(EXIT_FAILURE);
        }
        for (int i = 0; i < POP_SIZE; i++) {
            fscanf(file, "%lf", &population[i].fitness);
        }
        fclose(file);

        // Find the best individual
        double current_best_fitness = -1.0;
        int best_idx = -1;
        for (int i = 0; i < POP_SIZE; i++) {
            if (population[i].fitness > current_best_fitness) {
                current_best_fitness = population[i].fitness;
                best_idx = i;
            }
        }

        printf("Best Fitness: %.4f\n", current_best_fitness);

        // Check for stagnation
         if (current_best_fitness == best_fitness) {
        stagnation_count++;
        printf("No improvement in generation %d. Stagnation count: %d\n", generation, stagnation_count);
        if (stagnation_count >= MAX_STAGNATION) {
            printf("Stopping early due to stagnation: No improvement for %d generations.\n", MAX_STAGNATION);
            break;
        }
    } else {
        best_fitness = current_best_fitness;
        stagnation_count = 0;
    }

        // Preserve elite
        new_population[0] = population[best_idx];

        // Perform crossover and mutation
        cudaMemcpy(d_population, h_population, POP_SIZE * CHROMOSOME_LENGTH * sizeof(int), cudaMemcpyHostToDevice);
        crossover_kernel<<<blocks, BLOCK_SIZE>>>(d_population, d_new_population, POP_SIZE, CHROMOSOME_LENGTH);
        cudaDeviceSynchronize();
        mutation_kernel<<<blocks, BLOCK_SIZE>>>(d_new_population + CHROMOSOME_LENGTH, d_states, MUTATION_RATE, POP_SIZE - 1, CHROMOSOME_LENGTH);
        cudaDeviceSynchronize();

        // Copy results back
        cudaMemcpy(h_population, d_new_population, POP_SIZE * CHROMOSOME_LENGTH * sizeof(int), cudaMemcpyDeviceToHost);

        for (int i = 1; i < POP_SIZE; i++) {
            memcpy(population[i].genes, &h_population[i * CHROMOSOME_LENGTH], CHROMOSOME_LENGTH * sizeof(int));
        }
        population[0] = new_population[0];
    }

    // Cleanup
    cudaFree(d_population);
    cudaFree(d_new_population);
    cudaFree(d_states);
    free(h_population);
    free(population);
    free(new_population);

    return 0;
}



Overwriting cudamain.cu


In [ ]:
!nvcc -o cudamain cudamain.cu

cudamain.cu(37): warning #177-D: variable "gene_idx" was declared but never referenced
          int gene_idx = idx % chromosome_length;
              ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"



In [ ]:
import time

t = time.time()
!./cudamain


=== Generation 0 ===
Population Shape: (80, 7)
Chromosome 1: [0 1 1 1 1 1 1]
Selected Features: [1 2 3 4 5 6]
Chromosome 1 Accuracy: 0.9893939393939394
Chromosome 2: [0 0 1 1 0 1 0]
Selected Features: [2 3 5]
Chromosome 2 Accuracy: 0.6833333333333333
Chromosome 3: [1 0 1 1 1 0 0]
Selected Features: [0 2 3 4]
Chromosome 3 Accuracy: 0.9166666666666666
Chromosome 4: [0 1 1 1 1 0 0]
Selected Features: [1 2 3 4]
Chromosome 4 Accuracy: 0.9257575757575758
Chromosome 5: [1 1 1 0 1 0 1]
Selected Features: [0 1 2 4 6]
Chromosome 5 Accuracy: 0.9878787878787879
Chromosome 6: [1 1 0 0 1 1 1]
Selected Features: [0 1 4 5 6]
Chromosome 6 Accuracy: 0.9621212121212122
Chromosome 7: [0 1 1 0 0 0 1]
Selected Features: [1 2 6]
Chromosome 7 Accuracy: 0.8257575757575758
Chromosome 8: [0 0 0 0 1 0 1]
Selected Features: [4 6]
Chromosome 8 Accuracy: 0.7848484848484848
Chromosome 9: [0 0 1 1 1 0 0]
Selected Features: [2 3 4]
Chromosome 9 Accuracy: 0.8484848484848485
Chromosome 10: [1 1 1 1 0 1 0]
Selected Featu

In [ ]:
delta_t = time.time()-t
print("Time taken for total execution of parallel code:", delta_t)
parallel_time = delta_t

Time taken for total execution of parallel code: 86.5455253124237
